In [1]:
import networkx as nx
import json
from datetime import datetime

def criar_arquivo_gephi_temporal(caminho_arquivo_jsonl, arquivo_saida_gexf):
    """
    Cria um arquivo .gexf temporal para ser importado no Gephi.

    As arestas (interações) terão atributos de 'stance' e 'tempo' (via start/end).
    """
    # 1. Inicializar o Grafo Direcionado
    G = nx.DiGraph()

    # --- INÍCIO DA CORREÇÃO ---
    # Adicionar metadados para indicar que o grafo é dinâmico
    # Isso é essencial para o Gephi reconhecer os dados temporais.
    G.graph['mode'] = 'dynamic'
    G.graph['timeformat'] = 'datetime' 
    # --- FIM DA CORREÇÃO ---

    # 2. Ler o arquivo JSONL e popular o grafo
    with open(caminho_arquivo_jsonl, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                tweet = json.loads(line)
                
                autor = tweet.get('user')
                stance = tweet.get('stance')
                timestamp_str = tweet.get('created_at_iso') # formato ISO8601

                if not autor or not stance or not timestamp_str:
                    continue
                
                # Validação extra do formato da data (opcional, mas recomendado)
                # Garante que a string está no formato correto que o Gephi espera.
                # O formato esperado é YYYY-MM-DDTHH:MM:SS
                valid_timestamp = datetime.fromisoformat(timestamp_str.replace("Z", "+00:00")).isoformat()

                # Adiciona o autor como nó (se já não existir)
                G.add_node(autor)
                
                mencoes = [m['username'] for m in tweet.get('mentions', [])]
                usuario_respondido = tweet.get('in_reply_to_user')
                if usuario_respondido:
                    mencoes.append(usuario_respondido)
                
                # 3. Adicionar arestas com atributos de stance e tempo
                for mencionado in set(mencoes):
                    if mencionado:
                        G.add_node(mencionado)
                        G.add_edge(
                            autor, mencionado,
                            stance=stance,
                            # Usamos o timestamp validado
                            start=valid_timestamp,
                            end=valid_timestamp
                        )
            except (json.JSONDecodeError, KeyError, ValueError) as e:
                print(f"Erro ao processar linha: {line.strip()}. Erro: {e}")
                continue


    print(f"Grafo temporal criado com {G.number_of_nodes()} nós e {G.number_of_edges()} arestas.")

    # 4. Salvar o grafo no formato GEXF
    try:
        # --- ALTERAÇÃO: Especificar a versão 1.2 do GEXF ---
        nx.write_gexf(G, arquivo_saida_gexf, version='1.2')
        print(f"Arquivo '{arquivo_saida_gexf}' criado com sucesso!")
    except Exception as e:
        print(f"Ocorreu um erro ao salvar o arquivo: {e}")

# --- Como usar a função ---
# Certifique-se que este arquivo existe e contém os dados no formato esperado
arquivo_de_tweets_classificados = 'tweets_classificados.jsonl' 
nome_do_arquivo_gephi_temporal = 'scapegoat_temporal_corrigido.gexf'

criar_arquivo_gephi_temporal(arquivo_de_tweets_classificados, nome_do_arquivo_gephi_temporal)

Grafo temporal criado com 5760 nós e 5212 arestas.
Arquivo 'scapegoat_temporal.gexf' criado com sucesso!
